**Machine Learning based NLP CHATBOT**

---

In [34]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**IMPORTING REQUIRED LIBRARIES**

---



In [35]:
# Importing required libraries
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle
import re
from nltk.corpus import words
from nltk.stem import PorterStemmer

import nltk
nltk.download('words')


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

**PREPROCESSING DATA AND TEXT**

---

In [36]:
# Loading and preprocessing of data
csv_file_path = "/content/drive/MyDrive/Conversation.csv"
df = pd.read_csv(csv_file_path)
questions = df['question'].tolist()
answers = df['answer'].tolist()

# Preprocessing of text
def preprocess_text(text):
    text = text.lower()
    # more preprocessing text/steps can be added
    return text

questions = [preprocess_text(q) for q in questions]


**TOKENIZATION**

---



In [37]:
# Tokenization and padding
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(questions)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)


**ENCODIG**

---



In [38]:
# Encoding the answers
lbl_encoder = LabelEncoder()
lbl_encoder.fit(answers)
encoded_answers = lbl_encoder.transform(answers)


**NEURAL NETWORK MODEL**

---



In [39]:
# Building a simple neural network model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(len(answers), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


**TRAINING THE MODEL**

---



In [40]:
# Training of the model
epochs = 150
history = model.fit(padded_sequences, np.array(encoded_answers), epochs=epochs)


Epoch 1/150
117/117 [==============================] - 1s 6ms/step - loss: 8.2402 - accuracy: 0.0030
Epoch 2/150
117/117 [==============================] - 1s 6ms/step - loss: 8.2182 - accuracy: 0.0043
Epoch 3/150
117/117 [==============================] - 1s 6ms/step - loss: 8.2101 - accuracy: 0.0035
Epoch 4/150
117/117 [==============================] - 1s 6ms/step - loss: 8.2001 - accuracy: 0.0048
Epoch 5/150
117/117 [==============================] - 1s 7ms/step - loss: 8.1830 - accuracy: 0.0059
Epoch 6/150
117/117 [==============================] - 1s 7ms/step - loss: 8.1638 - accuracy: 0.0059
Epoch 7/150
117/117 [==============================] - 1s 6ms/step - loss: 8.1457 - accuracy: 0.0059
Epoch 8/150
117/117 [==============================] - 1s 6ms/step - loss: 8.1282 - accuracy: 0.0059
Epoch 9/150
117/117 [==============================] - 1s 6ms/step - loss: 8.1106 - accuracy: 0.0059
Epoch 10/150
117/117 [==============================] - 1s 6ms/step - loss: 8.0926 - accura

**SAVING TRAINED MODEL WITH TOKENIZER AND LABEL ENCODER**

---



In [41]:
# Saving the trained model, tokenizer, and label encoder
model.save("/content/drive/MyDrive/chat_model")

with open('/content/drive/MyDrive/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/label_encoder.pickle', 'wb') as enc_file:
    pickle.dump(lbl_encoder, enc_file, protocol=pickle.HIGHEST_PROTOCOL)


**DEFINING GREETING CONVERSATION**

---

In [42]:
# Greetings Conversation
greet_inputs = ('hello', 'hi', 'greetings', 'sup', 'hola', 'namaste', 'hey',)
greet_responses = ['hi', 'hey', '*nods*', 'hi there', 'hello', 'hola', 'namaste', 'yes! sup?']


**CHATFLOW AND INTERACTION WITH THE BOT**

---



In [44]:
# Interaction with the Chatbot
print("Hello, This is your Bot!")
print("To exit the conversation type 'bye'")
while True:
    user_input = input("User: ")

    if user_input.lower() == "bye":
        break

    user_input = preprocess_text(user_input)

        # Check for greetings
    if any(greet_input in user_input for greet_input in greet_inputs):
        bot_response = np.random.choice(greet_responses)
    else:
      user_sequence = tokenizer.texts_to_sequences([user_input])
      user_padded = pad_sequences(user_sequence, truncating='post', maxlen=max_len)
      predicted_encoded_response = model.predict(user_padded)
      predicted_response = lbl_encoder.inverse_transform([np.argmax(predicted_encoded_response)])
      bot_response = predicted_response[0]

    print("ChatBot:", bot_response)

print("ChatBot: Bye-Byee, Take care <3")


Hello, This is your Bot!
To exit the conversation type 'bye'
User: bye
ChatBot: Bye-Byee, Take care <3


**CLASSIFICATION METRICS**

---



In [53]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from prettytable import PrettyTable


In [54]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_answers, test_size=0.2, random_state=42)


In [55]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to original labels
y_test_original = lbl_encoder.inverse_transform(y_test)
y_pred_original = lbl_encoder.inverse_transform(y_pred_classes)


24/24 [==============================] - 0s 3ms/step


In [56]:
# Print a classification report
print("Classification Report:\n", classification_report(y_test_original, y_pred_original))


Classification Report:
                                                                                    precision    recall  f1-score   support

                                             "Â…and you know you should be glad!"       1.00      1.00      1.00         1
                                     a blind date doesn't mean that she is blind!       1.00      1.00      1.00         1
                                                      a girl i met at the market.       1.00      1.00      1.00         1
                                      a lot, like the party i'm having on friday.       1.00      1.00      1.00         1
                                     a team of doctors gave her a whole new face.       0.00      0.00      0.00         1
                                         a third guy punched one of the two guys.       1.00      1.00      1.00         1
                                     about 50 percent of the time he's traveling.       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [57]:
# Calculate and print classification metrics in a table
metrics_table = PrettyTable()
metrics_table.field_names = ["Metric", "Value"]

metrics_table.add_row(["Accuracy", accuracy_score(y_test_original, y_pred_original)])
metrics_table.add_row(["Precision", precision_score(y_test_original, y_pred_original, average='weighted')])
metrics_table.add_row(["Recall", recall_score(y_test_original, y_pred_original, average='weighted')])
metrics_table.add_row(["F1 Score", f1_score(y_test_original, y_pred_original, average='weighted')])

print("\nMetrics Table:")
print(metrics_table)


Metrics Table:
+-----------+--------------------+
|   Metric  |       Value        |
+-----------+--------------------+
|  Accuracy | 0.8080536912751678 |
| Precision | 0.7992323671853871 |
|   Recall  | 0.8080536912751678 |
|  F1 Score | 0.7968823354382253 |
+-----------+--------------------+


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
